# Отчет об участии в соревновании Rekko challenge

Александр Шулков

## Постановка задачи

Соревнование проводится онлайн-кинотеатром, в качестве исходных данных предоставлены:
 - история просмотров фильмов (покупок и просмотров по подписке) пользователями примерно за полгода (таблица `transactions`)
 - история проставления ими рейтингов за тот же период (таблица `ratings`)
 - история добавления пользователями фильмов в избранное (таблица `bookmarks`)
 - каталог фильмов с обезличенной метаинформацией (таблица `catalogue`)

В соревновании требуется предсказать 20 фильмов, которые посмотрит каждый пользователь из тестовой выборки за следующие 60 дней. Тестовая выборка включает в себя 50000 пользователей, при этом только 13251 из них присутствует в таблице рейтингов. В таблице просмотров присутствуют практически все (49992) пользователи из тестового множества.

## Вариант решения

Для восстановления недостающих рейтингов используем один из стандартных подходов - SVD-разложение. Из-за большого числа пользователей (500000) применение user-based коллаборативной фильтрации представляется вычислительно нереализуемым (для расчёта попарных расстояний потребуется рассчитать $500000 * 500000 / 2 = 1.25e^{11}) $). Применение item-based подхода выглядит более реалистичным вариантом, но всё же требует больших вычислений ($10200 * 10200 / 2 = 5.2e^7$ расстояний).

Поскольку явно проставленных пользователями рейтингов гораздо меньше, чем данных о просмотренных ими фильмах, хочется использовать эту неявную информацию для обогащения данных о предпочтениях пользователей. Например, продолжительность просмотра фильма кажется хорошей характеристикой того, понравился ли фильм пользователю. 

На основе данных о просмотрах, добавлении фильма в избранное, а также части метаинформации о фильме построим регрессионную модель, с помощью которой будем предсказывать рейтинги для пар "пользователь-фильм", встречающихся в таблице просмотров, но отсутствующих в таблице рейтингов. В качестве дополнительного признака будем использовать сконструированный: отношение времени просмотра к длительности фильма (`watched_ratio`). 

Для решения задачи регрессии используем алгоритм градиентного бустинга (XGBRegressor) как один из наиболее эффективных и не требующий существенной предобработки данных. Настраиваем его, минимизируя квадратичную ошибку в предсказании рейтингов.

Предсказанные таким образом рейтинги добавляются к исходной таблице. На этих данных строится матрица рейтингов, для которой выполняется SVD-разложение с параметрами по умолчанию (на 6 компонент). По результатам разложения восстанавливаются все недостающие рейтинги для тестовых пользователей. Для каждого пользователя выбирается топ-500 фильмов с максимальным прогнозируемым рейтингом (из таблицы транзакций видно, что почти все пользователи из тестового множества просмотрели менее 500 фильмов), из которых удаляются ранее просмотренные фильмы и выбирается топ-20.

## Эксперименты

- предсказание фильмов из топ-500 по общему количеству рейтингов для всех пользователей (с фильтрацией по уже просмотренным) - дало относительно неплохой результат (0.0188)
- использование записей из `bookmarks` для дополнения таблицы рейтингов - не дало положительных результатов 
- выбор подходящего алгоритма для решения задачи регрессии (сравнивались RandomForestRegressor, GradientBoostingRegressor и XGBRegressor), подбор параметров
- попытки использования алгоритмов SVD и SVD++ из пакета **surprise** - не дали каких-либо положительных результатов



## Что ещё можно было бы сделать

- провести нормализацию рейтингов
- реализовать item-based коллаборативную фильтрацию - как на данных рейтингов, так и на истории просмотров
- для расчёта расстояний между фильмами задействовать в модели признаки из каталога фильмов,  таким образом можно было бы предлагать пользователям фильмы, которые отсутствуют в таблицах рейтингов и истории просмотров (1530 фильмов)
- в частности, поле `attributes` - по всей видимости, в нём содержится информация о режиссёрах и актёрах фильма, которая могла бы быть полезна для определения схожести между фильмами
- подобрать параметры SVD-разложения (количество компонент) по метрике RMSE / MAE качества предсказания существующих рейтингов
- разбить данные на обучающую и тестовую выборки по времени (по полю `ts`) и реализовать полноценный подбор параметров алгоритмов с оптимизацией непосредственно целевой метрики соревнования (MNAP@20) 


## Итоговый результат

Лучший результат - 0.02615, место в лидерборде - 104, ник - AlexanderShulkov

 



In [1]:
# загружаем необходимые библиотеки
import os
import json
import pandas as pd
import numpy as np
import tqdm

from collections import defaultdict

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score

from xgboost import XGBRegressor

from scipy.sparse import coo_matrix, csr_matrix

import seaborn as sns

import gc

%matplotlib inline
sns.set()

import warnings
warnings.filterwarnings("ignore")

# задаём пути к данным
DATA_PATH = './data/'
PREDICTIONS_PATH = './predictions/'

### Загрузка данных

In [6]:
# -- каталог фильмов
catalogue = pd.read_json(
    os.path.join(DATA_PATH, 'catalogue.json'), 
    orient = 'index', 
    typ = 'frame'
    )

# -- история просмотров
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

# -- история добавления в избранное
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

# -- рейтинги
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.float64
    }
)

# -- множество тестовых пользователей
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

### Предобработка данных

In [7]:
# удаляем неиспользуемые в дальнейшем столбцы, преобразуем категориальные переменные
catalogue.index.rename('element_uid', inplace = True)
catalogue.drop(['attributes', 'availability'], axis = 1, inplace = True)
catalogue = pd.get_dummies(catalogue, columns = ['type'])

transactions = pd.get_dummies(transactions, columns = ['consumption_mode'])

bookmarks.rename({'ts': 'ts_bookmark'}, axis=1, inplace = True)

In [8]:
# дополняем таблицу просмотров признаками из других таблиц
transactions = transactions.merge(catalogue, 
                                  how = 'left', 
                                  left_on = 'element_uid', right_index = True
                                 )

transactions = transactions.merge(bookmarks, 
                                  how = 'left', 
                                  left_on = ['element_uid', 'user_uid'], 
                                  right_on =  ['element_uid', 'user_uid']
                                 )

transactions = transactions.merge(ratings.drop('ts', axis = 1),
                                  how = 'left',
                                  left_on = ['element_uid', 'user_uid'],
                                  right_on =  ['element_uid', 'user_uid']
                                  )

In [9]:
# создаём новые признаки
transactions['watched_ratio'] = (transactions.watched_time)/((transactions.duration + 5)*60)

transactions['is_bookmarked'] = transactions.ts_bookmark.notnull().astype(int)

transactions['ts_difference'] = transactions.ts - transactions.ts_bookmark
transactions.loc[transactions['ts_difference'].isnull(), 'ts_difference'] = 0

### Обучение регрессионной модели

In [13]:
# формируем обучающую выборку
train = transactions[transactions.rating.notnull()]
test = transactions[transactions.rating.isnull()]
print (train.shape, test.shape)

X_train = train.drop(['rating', 'element_uid', 'user_uid', 'ts', 'ts_bookmark'], axis = 1)
y_train = train['rating']

X_test = test.drop(['rating', 'element_uid', 'user_uid', 'ts', 'ts_bookmark'], axis = 1)

(357343, 23) (9285669, 23)


In [14]:
X_train.head(3)

,watched_time,device_type,device_manufacturer,consumption_mode_P,consumption_mode_R,consumption_mode_S,duration,feature_1,feature_2,feature_3,feature_4,feature_5,type_movie,type_multipart_movie,type_series,watched_ratio,is_bookmarked,ts_difference
0,4282,0,50,0,0,1,90,4.166108e+07,0.739609,45,1.141929,0.654707,1,0,0,0.751228,0,0.0
64,1529,5,90,0,0,1,30,4.118513e+07,0.692949,16,1.140273,0.592716,1,0,0,0.728095,0,0.0
93,4867,0,50,0,0,1,80,4.029988e+07,0.677350,12,1.131807,0.449667,1,0,0,0.954314,0,0.0


In [15]:
# подбираем оптимальные параметры алгоритма по метрике MSE
cv = KFold(n_splits=5, shuffle=True, random_state=42)

for n in [200, 250, 300]:
    for depth in [2, 3, 5]:
        for sample in [0.6, 0.8, 1.0]:
            clf = XGBRegressor(n_estimators = n, max_depth = depth, learning_rate = 0.1, subsample = sample)
            res = cross_val_score(clf, X_train, y_train, scoring = 'neg_mean_squared_error', cv = cv, n_jobs = 3)
            print(n, depth, sample, res, res.mean())

200 2 0.6 [-3.49551283 -3.51947782 -3.55423512 -3.52164109 -3.5218132 ] -3.5225360105466925
200 2 0.8 [-3.50030856 -3.51961528 -3.554773   -3.52219569 -3.52204094] -3.5237866942909486
200 2 1.0 [-3.50008275 -3.51939921 -3.5548448  -3.52505776 -3.52177648] -3.524232200246577
200 3 0.6 [-3.45135793 -3.4752408  -3.5109287  -3.47585196 -3.479687  ] -3.4786132796260154
200 3 0.8 [-3.4512768  -3.47702988 -3.51152794 -3.47642674 -3.48307469] -3.4798672124917074
200 3 1.0 [-3.45335826 -3.47820154 -3.51505828 -3.47892526 -3.48237577] -3.481583823118931
200 5 0.6 [-3.39915468 -3.42722166 -3.4635902  -3.42456228 -3.43651156] -3.430208077060674
200 5 0.8 [-3.40001603 -3.4252415  -3.46020192 -3.42528112 -3.43231488] -3.4286110920139046
200 5 1.0 [-3.39738158 -3.42303501 -3.46355729 -3.42679766 -3.43555204] -3.429264713612613
250 2 0.6 [-3.48807803 -3.5107691  -3.54676502 -3.51350412 -3.51230763] -3.5142847800751236
250 2 0.8 [-3.49305607 -3.51159584 -3.54652886 -3.51599783 -3.51442495] -3.516320708

In [36]:
# обучаем алгоритм
xgb = XGBRegressor(n_estimators = 300, max_depth = 5, learning_rate = 0.1, subsample = 0.8)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

In [33]:
# можем проанализировать, какие признаки показались алгоритму наиболее полезными
pd.DataFrame(zip(X_train.columns, xgb.feature_importances_), 
             columns = ['feature', 'importance']
            ).sort_values('importance', ascending = False)

,feature,importance
8,feature_2,0.314476
11,feature_5,0.209613
7,feature_1,0.050264
9,feature_3,0.044602
6,duration,0.039575
1,device_type,0.035492
0,watched_time,0.034932
2,device_manufacturer,0.034611
5,consumption_mode_S,0.030966
12,type_movie,0.029722


In [19]:
# предсказываем недостающие рейтинги
pred_xgb = xgb.predict(X_test)

### Формирование обогащенной матрицы рейтингов

In [20]:
# формируем дополненую таблицу рейтингов
predicted_ratings = test[['user_uid', 'element_uid']].copy()
predicted_ratings['rating'] = pred_xgb
predicted_ratings['ts'] = 0.0

ratings_full = pd.concat((ratings, predicted_ratings))
print(ratings_full.shape)

(9724459, 4)


In [21]:
# перенумеровываем ID пользователей и фильмов для создания разреженной матрицы 
enc_user = LabelEncoder()
enc_movie = LabelEncoder()

enc_user = enc_user.fit(ratings_full.user_uid.values)
enc_movie = enc_movie.fit(ratings_full.element_uid.values)

In [22]:
# создаём матрицу рейтингов 
R = coo_matrix((ratings_full.rating.values, 
                (
                    enc_user.transform(ratings_full.user_uid.values), 
                    enc_movie.transform(ratings_full.element_uid.values)
                )
               ))
R = R.tocsr()
R

<500000x8670 sparse matrix of type '<class 'numpy.float64'>'
	with 9724459 stored elements in Compressed Sparse Row format>

### SVD разложение и предсказание рейтингов

In [23]:
# выполняем матричное разложение
from scipy.sparse.linalg import svds
u, s, vt = svds(R, k=6)
s_matrix = np.diag(s)
print (u.shape, s.shape, vt.shape)

(500000, 6) (6,) (6, 8670)


In [26]:
# рассчитываем рейтинги по всем фильмам для тестовых пользователей
predictions = np.dot(np.dot(u[enc_user.transform(list(test_users))], s_matrix), vt)

### Формирование ответов

In [27]:
# сортируем рейтинги, определяем 500 фильмов с самыми высокими значениями (для каждого пользователя) 
predictions_sorted_desc_500 = np.fliplr(np.argsort(predictions, axis=1))[:,:500]

In [28]:
# заполняем словарь ID фильмов из топ-500 
top500 = {}

for i, user_uid in enumerate(tqdm.tqdm(test_users)):
    top500[user_uid] = list(enc_movie.inverse_transform(predictions_sorted_desc_500[i]))

100%|██████████| 50000/50000 [00:26<00:00, 1882.63it/s]


In [29]:
# заполняем словарь уже просмотренных пользователями фильмов
filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

100%|██████████| 9643012/9643012 [00:18<00:00, 513152.85it/s]


In [30]:
# фильтруем просмотренные фильмы, оставляем первые 20
result = {}
for user_uid in tqdm.tqdm(test_users):
    result[user_uid] = [int(x) for x in top500[user_uid] if x not in filtered_elements[user_uid]][:20]

100%|██████████| 50000/50000 [00:56<00:00, 879.34it/s]


In [31]:
# выгружаем ответ в файл
with open(os.path.join(PREDICTIONS_PATH, 'svd_by_ratings_xgb_filtered.json'), 'w') as f:
    json.dump(result, f)